In [3]:
import requests,json
from config import *
import pandas as pd
import numpy as np
import time

## API documentation: https://www.robotevents.com/api/v2

In [1]:
def get_events_by_sku(event_sku):
    # sample sku:  "RE-VRC-23-4112"
    url = base_url + "/events?"+"sku[]="+event_sku+"&myEvents=false"
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}    
    r = requests.request("GET",url, headers=headers, data=payload)
    return r.json()['data']


def get_teams_by_event_id(event_id):
    url = base_url + "/events/"+ event_id +"/teams"
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}    
    r = requests.request("GET",url, headers=headers, data=payload)
    return r.json()['data']

def get_teams_by_state(state):
    url = 'https://www.robotevents.com/api/v2/teams?registered=true&program%5B%5D=1&grade%5B%5D=Middle%20School&country%5B%5D=US&myTeams=false'
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}    
    r = requests.request("GET",url, headers=headers, data=payload)    
    return r.json()['data']

def get_events_by_team(event_id,season):
    url = base_url + "/teams/"+ event_id+"/events/?season%5B%5D="+season
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}    
    r = requests.request("GET",url, headers=headers, data=payload)
    return r.json()['data']  

def get_team_id_by_number(number):
    url = base_url + "/teams?number%5B%5D="+ number +"&registered=true&program%5B%5D=1&grade%5B%5D=Middle%20School&country%5B%5D=US&myTeams=false"
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}    
    r = requests.request("GET",url, headers=headers, data=payload)
    return r.json()['data'][0]    

def get_awards_by_team(team_id):
    url = base_url + "/teams/"+ team_id +"/awards"
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}    
    r = requests.request("GET",url, headers=headers, data=payload)
    return r.json()['data']   

def format_awards(awards):
    list_awards = []
    for awd in awards:
        summary = awd['title']+"       "+str(awd['event']['id'])+awd['event']['name']
        list_awards.append(summary)    
    return list_awards

def get_skill_by_team(team_id,skill_type='driver',season='181'):
    #skill_type:  driver, programming
    url = base_url +'/teams/'+team_id +'/skills?type%5B%5D='+skill_type+'&season%5B%5D='+ season
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}
    try:
        r = requests.request("GET",url, headers=headers, data=payload)
        return r.json()['data'] 
    except:
        return([])
     

def format_skill(list_skill):
    max_score = 0
    venue = ''
    for skill in list_skill:
        if skill['score']>max_score:
            max_score = skill['score']
            venue = skill['event']['name']
    return(str(max_score))

def get_matches_by_team_and_event(event_id,team_id,div="1"):
    url = base_url + "/events/"+ event_id +"/divisions/"+div+"/matches?team%5B%5D="+team_id
    headers = { 
        "Authorization" : token,
        "Accept": "application/json",
        'Content-Type': 'application/json'
    }
    payload = {}    
    r = requests.request("GET",url, headers=headers, data=payload)
    return r.json()['data']

def format_match(list_matches):
    matches_of_season = []
    if list_matches != []:
        for match in list_matches:
            if match["started"] is not None:
                combat = match['name']+"    "+str(match['event']['id'])+"    "+match["event"]["name"]+ "    "+match["started"]
                matches_of_season.append(combat)
        return(matches_of_season)
    else:
        return([])

In [3]:
list_maryland_middle = ['9290B','23382A','18990A','9290A','5769B','5769W','12580A','7135Z','12768A','7135X','9290C','75368A','7135B','80018V','47418B','5769C','99976A','7135P','6537A','5769A','5770C','99976B','9316C','9012A','7135M','99330A','6537B','80018W','99976C','6750B']

list_teams = []
for t_number in list_maryland_middle:
    t_info = get_team_id_by_number(t_number)
    list_teams.append(t_info)
    
df_teams = pd.DataFrame(list_teams)
df_teams

,id,number,team_name,robot_name,organization,location,registered,program,grade
0,4406,9290B,Hereford Zone Robotics,Sackbot,Hereford Middle School,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
1,155425,23382A,Fifth Dimension,,Weestar,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
2,155702,18990A,Monkeys Robotics,None,ICBF,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
3,4401,9290A,Hereford Zone Robotics,FN-2187,HEREFORD MIDDLE,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
4,35107,5769B,Disco Ducks,,PICCOWAXEN MIDDLE SCHOOL,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
5,144840,5769W,Rizzinators,,Piccowaxen Middle School,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
6,155696,12580A,OctoBots Robotics,None,AOE Robotics Club,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
7,128429,7135Z,Loyola Robotics - Z,The DONminator,Loyola Blakefield,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
8,155695,12768A,Tigers Robotics,None,AOE Robotics Club,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School
9,128427,7135X,Loyola Robotics-X,,Loyola Blakefield,"{'venue': None, 'address_1': '', 'address_2': ...",True,"{'id': 1, 'name': 'VEX Robotics Competition', ...",Middle School


In [4]:

#df_team_results = pd.DataFrame()
df_team_results = df_teams.copy(True)
df_team_results['awards']=''
df_team_results['driver_skill']=''
df_team_results['programming_skill']=''
df_team_results['total_skill']=''
df_team_results['matches_this_season']=''
                
for index, row in df_team_results.iterrows():
        time.sleep(3)
        awards = get_awards_by_team(str(row['id']))
        df_team_results.at[index,'awards'] = format_awards(awards)

        list_driver_skill= get_skill_by_team(str(row['id']),'driver','181')
        driver_skill = format_skill(list_driver_skill)
        df_team_results.at[index,'driver_skill'] = driver_skill

        list_programming_skill= get_skill_by_team(str(row['id']),'programming','181')
        programming_skill = format_skill(list_programming_skill)
        df_team_results.at[index,'programming_skill'] = programming_skill

        df_team_results.at[index,'total_skill'] =int(driver_skill)+int(programming_skill)

        events_this_season = get_events_by_team(str(row['id']),'181')
        list_event_id=[]
        list_matches_of_season = []
        for ev in events_this_season:
            list_event_id.append(ev['id'])
        for event_id in list_event_id:
            time.sleep(3)
            list_matches = get_matches_by_team_and_event(str(event_id),str(row['id']),div="1")
            if list_matches != []:
                list_matches_of_season = list_matches_of_season + format_match(list_matches)
        df_team_results.at[index,'matches_this_season'] = list_matches_of_season

df_team_results

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


JSONDecodeError: [Errno Expecting value] <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .big-button,body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{backgrou

In [5]:
df_team_results.to_excel('2024_Maryland_VEX_MiddleSchool_Teams.xlsx')

In [4]:
 t_info = get_team_id_by_number("23382A")

In [5]:
t_info

{'id': 155425,
 'number': '23382A',
 'team_name': 'Fifth Dimension',
 'robot_name': '',
 'organization': 'Weestar',
 'location': {'venue': None,
  'address_1': '',
  'address_2': None,
  'city': 'North Potomac',
  'region': 'Maryland',
  'postcode': '20878',
  'country': 'United States',
  'coordinates': {'lat': 39.1, 'lon': -77.2}},
 'registered': True,
 'program': {'id': 1, 'name': 'VEX Robotics Competition', 'code': 'VRC'},
 'grade': 'Middle School'}

In [6]:
print(get_events_by_sku("RE-VRC-23-2633"))

[{'id': 52633, 'sku': 'RE-VRC-23-2633', 'name': '2023-2024 Maryland VEX Robotics Competition (VRC)  Middle School State Championship - Over Under', 'start': '2024-02-17T00:00:00-05:00', 'end': '2024-02-17T00:00:00-05:00', 'season': {'id': 181, 'name': 'VRC 2023-2024: Over Under', 'code': None}, 'program': {'id': 1, 'name': 'VEX Robotics Competition', 'code': 'VRC'}, 'location': {'venue': 'Hereford Middle School', 'address_1': '712 Corbett Road', 'address_2': None, 'city': 'Monkton', 'region': 'Maryland', 'postcode': '21111', 'country': 'United States', 'coordinates': {'lat': 39.57273, 'lon': -76.653274}}, 'locations': {'2024-02-17': {'venue': 'Hereford Middle School', 'address_1': '712 Corbett Road', 'address_2': None, 'city': 'Monkton', 'region': 'Maryland', 'postcode': '21111', 'country': 'United States', 'coordinates': {'lat': 39.57273, 'lon': -76.653274}}}, 'divisions': [{'id': 1, 'name': 'Default Division', 'order': 1}], 'level': 'Regional', 'ongoing': False, 'awards_finalized': F